In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from functiona import *

In [2]:
# デバイスの割り当て

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:

# 1階層
transform1 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
    transforms.Lambda(lambda x: x.view(-1)),
])

# 3階層
transform2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5),
])

# データセットの取得
data_root = './hidden_data/'

# 一階層セット
#　訓練データ
train_set1 = datasets.CIFAR10(root = data_root,
                              train = True,
                              download = True,
                              transform = transform1)

test_set1 = datasets.CIFAR10(root = data_root,
                              train = False,
                              download = True,
                              transform = transform1)

# ３階層
train_set2 = datasets.CIFAR10(root = data_root,
                              train = True,
                              download = True,
                              transform = transform2)

test_set2 = datasets.CIFAR10(root = data_root,
                              train = False,
                              download = True,
                              transform = transform2)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [4]:
# データローダの定義
batch_size = 100

# 1階層
train_loader1 = DataLoader(dataset=train_set1,
                           batch_size=batch_size,
                           shuffle=True)

test_loader1 = DataLoader(dataset=test_set1,
                           batch_size=batch_size,
                           shuffle=False)

#　2階層
train_loader2 = DataLoader(dataset=train_set2,
                           batch_size=batch_size,
                           shuffle=True)

test_loader2 = DataLoader(dataset=test_set2,
                           batch_size=batch_size,
                           shuffle=False)

In [14]:
# データローダの確認
for images1, labels1 in train_loader1:
    break

for images2, labels2 in train_loader2:
    break

print(images1[0].shape)
print(images2.shape)

torch.Size([3072])
torch.Size([100, 3, 32, 32])


In [6]:
# 正解ラベル定義
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [15]:
n_input = images1[0].view(-1).shape[0]

n_output = len(classes)

n_hidden = 128

# 結果確認
print(f'n_input: {n_input}  n_hidden: {n_hidden} n_output: {n_output}')

n_input: 3072  n_hidden: 128 n_output: 10


In [16]:
# モデル

class Net(nn.Module):
    def __init__(self, n_input, n_output, n_hidden):
        super().__init__()

        self.l1 = nn.Linear(n_input, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_output)
        self.relu = nn.ReLU(inplace=True)


    def forward(self, x):
        x1 = self.l1(x)
        x2 = self.relu(x1)
        x3 = self.l2(x2)
        return x3

In [18]:

class CNN(nn.Module):
    def __init__(self, n_output, n_hidden):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d((2, 2))
        self.flatten = nn.Flatten()
        self.l1 = nn.Linear(6272, n_hidden)
        self.l2 = nn.Linear(n_hidden, n_output)

        self.features = nn.Sequential(
            self.conv1,
            self.relu,
            self.conv2,
            self.relu,
            self.maxpool
        )

        self.classifier = nn.Sequential(
            self.l1, 
            self.relu,
            self.l2
        )

    def forward(self, x):
        x1 = self.features(x)
        x2 = self.flatten(x1)
        x3 = self.classifier(x2)
        return x3       

In [19]:
net = CNN(n_output, n_hidden).to(device)
criterion = nn.CrossEntropyLoss()
lr = 0.01
optimizer = optim.SGD(net.parameters(), lr)

In [21]:
# cnnの実装
start = time.time()

torch.seed()

net = CNN(n_output, n_hidden).to(device)

criterion = nn.CrossEntropyLoss()

lr = 0.01
optimizer = optim.SGD(net.parameters(), lr)

num_epochs = 50

history2 = np.zeros((0, 5))

history2 = fit(net, optimizer, criterion, num_epochs, train_loader2, test_loader2, device, history2)

end = time.time()

print(f'掛かった時間:{end - start}')

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch [1/50], loss: 2.03958 acc: 0.26982 val_loss: 1.82522, val_acc: 0.36500


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch [2/50], loss: 1.75475 acc: 0.38134 val_loss: 1.68791, val_acc: 0.40470


  0%|          | 0/500 [00:00<?, ?it/s]

...

Epoch [48/50], loss: 0.02081 acc: 0.99926 val_loss: 1.92505, val_acc: 0.66170


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch [49/50], loss: 0.01807 acc: 0.99920 val_loss: 1.95827, val_acc: 0.65650


  0%|          | 0/500 [00:00<?, ?it/s]

Epoch [50/50], loss: 0.01483 acc: 0.99960 val_loss: 1.98318, val_acc: 0.65890
掛かった時間:770.2849364280701


・CPU(macbook pro intel i5)では -> 掛かった時間:2763.5931901931763   
・GPU(colab:T4)では -> 掛かった時間:921.6939368247986  
・GPU(MyPC:RTX3080)では -> 掛かった時間:770.2849364280701 